In [ ]:
import numpy as np

# Load the data and labels from the CSV files
data = np.loadtxt('train_data.csv', delimiter=',')
labels = np.loadtxt('train_labels.csv', delimiter=',')
np.shape(data)

# order sample randomly
order = list(range(np.shape(data)[0]))
np.random.shuffle(order)
train = data[order, :]
traint = labels[order, :]


# This code is inspired from Stephen Marsland
class MLP:
    """ A Multi-Layer Perceptron"""

    def __init__(self, inputs, targets, number_of_nodes_in_hidden_layer):
        """ Constructor """
        # Set up network size
        self.nIn = np.shape(inputs)[1]  # return col: number of nodes in Input layer
        self.nOut = np.shape(targets)[1]  # return col: number of nodes in Output layer
        self.nHidden = number_of_nodes_in_hidden_layer  # number of nodes in Hidden layer

        # Initialise random scaled weights to weights 1 & weights 2
        self.weights1 = (np.random.rand(self.nIn + 1, self.nHidden) - 0.5) * 2 / np.sqrt(self.nIn)
        self.weights2 = (np.random.rand(self.nHidden + 1, self.nOut) - 0.5) * 2 / np.sqrt(self.nHidden)

    def train(self, inputs, targets, eta, n_iterations):
        # eta = learning rate
        # Add bias of -1 array into input
        inputs = np.concatenate((inputs, -np.ones((np.shape(inputs)[0], 1))), axis=1)

        for n in range(n_iterations):
            # Run Forward
            # inputs(x) - weights1 -> (self.hidden) hidden (self.hidden + bias) -> weights2 -> output
            outputs = self.run_fwd(inputs)

            # Start Backpropagation
            # Cross-Entropy Loss Function
            cost = -np.mean(targets * np.log(outputs + 1e-8))

            # Error function 1/2*sum(t - o)^2
            error = 0.5 * np.sum((targets - outputs) ** 2)

            # Find accuracy of the output
            accuracy = self.check_accuracy(outputs, targets)

            # Print results at end of each iteration
            print("\nITERATION ", n, "\nACCURACY = ", accuracy)

            # Print overall results at end of nth iteration
            if n == n_iterations - 1:
                print("\n\nOVERALL RESULTS: \nMAX ACCURACY OBTAINED: ", accuracy, "%")
                print("\nMAX ACCURACY OUTPUT: ", self.convert_to_one_hot_output(outputs))

            # error signal output = o(1-o)(t-o)
            delta_out = outputs * (1.0 - outputs) * (targets - outputs)

            # w_new = w_old + eta*delta*o  # updating new weights to weights 2
            self.weights2 += eta * (np.dot(np.transpose(self.hidden_sigmoid), delta_out))

            # o(1-o)w*deltao
            delta_hidden = self.hidden_sigmoid * (1.0 - self.hidden_sigmoid) * (
                np.dot(delta_out, np.transpose(self.weights2)))

            # updating new weights to weights 1
            self.weights1 += eta * (np.dot(np.transpose(inputs), delta_hidden[:, :-1]))

    def run_fwd(self, inputs):
        """ Run the network forward """
        # inputs(x) - weights1 -> (self.hidden) hidden (self.hidden + bias) -> weights2 -> output
        # x.w
        self.hidden = np.dot(inputs, (self.weights1))

        # Run through activation function 1/(1 + e^-(B*x.w))
        # o1,o2,on ...
        self.hidden_sigmoid = self.sigmoid(self.hidden)

        # add bias nodes (column)
        self.hidden_sigmoid = np.concatenate((self.hidden_sigmoid, -np.ones((np.shape(inputs)[0], 1))), axis=1)

        # o = o_h . w
        outputs = np.dot(self.hidden_sigmoid, self.weights2)

        # Final output after activation function
        output_sigmoid = self.sigmoid(outputs)

        return output_sigmoid

    def check_accuracy(self, calculated_outputs, targets):
        predicted_classes = np.argmax(calculated_outputs, axis=1)
        # Number of classes
        num_classes = calculated_outputs.shape[1]
        # Create one-hot encoded representation
        one_hot = np.zeros((calculated_outputs.shape[0], num_classes))
        one_hot[np.arange(calculated_outputs.shape[0]), predicted_classes] = 1
        correctness = np.sum(np.equal(one_hot, targets)) / np.size(one_hot)
        return correctness * 100

    def convert_to_one_hot_output(self, calculated_outputs):
        predicted_classes = np.argmax(calculated_outputs, axis=1)
        # Number of classes
        num_classes = calculated_outputs.shape[1]
        # Create one-hot encoded representation
        one_hot = np.zeros((calculated_outputs.shape[0], num_classes))
        one_hot[np.arange(calculated_outputs.shape[0]), predicted_classes] = 1
        return one_hot

    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))


# Single Function to use the network to predict the test set
def train_and_predict(test_data):
    # Train the MLP
    net = MLP(train, traint, 60)  # MLP(inputs, targets, number_of_nodes_in_hidden_layer)
    net.train(train, traint, 0.0001, 50)  # train(inputs, targets, eta, n_iterations)

    # Predict the test set
    test_data_with_bias = np.concatenate((test_data, -np.ones((np.shape(test_data)[0], 1))), axis=1)
    predicted_outputs = net.run_fwd(test_data_with_bias)
    predicted_classes = np.argmax(predicted_outputs, axis=1)

    # Convert predicted classes to one-hot encoded representation
    num_classes = predicted_outputs.shape[1]
    one_hot_predictions = np.zeros((predicted_outputs.shape[0], num_classes))
    one_hot_predictions[np.arange(predicted_outputs.shape[0]), predicted_classes] = 1
    return one_hot_predictions


train_and_predict(train)
